In [13]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [36]:
url_inicial = 'http://trace.org.mx/index.php/trace/issue/archive?issuesPage=3#issues'
url_root = 'http://trace.org.mx/index.php/trace/issue/archive?issuesPage=3#issues'
r=requests.get(url_inicial)

In [37]:
soup = BeautifulSoup(r.text, 'html.parser')

In [38]:
lista_articulo = soup.find_all('div', style='float: left; width: 100%;')
links_libros = [x.find('a').get('href') for x in lista_articulo]
links_libros = [urljoin (url_root, i) for i in links_libros]
links_libros

['http://trace.org.mx/index.php/trace/issue/view/Trace%20no.%2031',
 'http://trace.org.mx/index.php/trace/issue/view/Trace%20no.%2030',
 'http://trace.org.mx/index.php/trace/issue/view/60']

In [39]:
def get_url_items(soup,url):
    lista_articulo = soup.find_all('div', style='float: left; width: 100%;')
    links_libros = [x.find('a').get('href') for x in lista_articulo]
    links_libros = [urljoin (url_root, i) for i in links_libros]
    return links_libros

In [40]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina http://trace.org.mx/index.php/trace/issue/archive?issuesPage=3#issues


In [41]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

3

In [42]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [43]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Trace (Travaux et Recherches dans les Amériques du Centre)'
    a='V. Ciencias Sociales'
    tem='Ciencias sociales y humanas'
    url='http://trace.org.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h2').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('div', id='issueCoverImage').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [44]:
list_scraper=list_scraper[0:13]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2


In [45]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Trace (Travaux et Recherches dans les Amérique...,V. Ciencias Sociales,Ciencias sociales y humanas,Núm. 31 (1997),None,http://trace.org.mx,http://trace.org.mx/index.php/trace/issue/arch...
1,Trace (Travaux et Recherches dans les Amérique...,V. Ciencias Sociales,Ciencias sociales y humanas,Núm. 30 (1996),None,http://trace.org.mx,http://trace.org.mx/index.php/trace/issue/arch...
2,Trace (Travaux et Recherches dans les Amérique...,V. Ciencias Sociales,Ciencias sociales y humanas,Núm. 01 (1978),None,http://trace.org.mx,http://trace.org.mx/index.php/trace/issue/arch...


In [46]:
df_catalogo.to_csv('Revista199.03.csv', index=False)